## Exploratory Data Analysis

5.11 Assignment 

Brad Howlett (bth2g)  
Eric Larson (rel4yx)  
Hanim Song (hs4cf) 
---

In [1]:
## this is for my (Eric) Spark setup, you don't need to run this
import findspark
findspark.init()

In [2]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.types as typ
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from functools import reduce
from pyspark.sql.functions import col, asc
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, DoubleType, DateType
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
 import pyspark.sql.column
 from pyspark.sql.functions import sum, count, avg, expr, lit

In [4]:
spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

## Utility functions

In [5]:
def csv_header(filename: str):
    with open(filename, 'r') as csv:
        line = csv.readline().rstrip()
    headers = line.split(',')
    return headers

In [6]:
csv_header('fec_data/cn/cn_header_file.csv')

['CAND_ID',
 'CAND_NAME',
 'CAND_PTY_AFFILIATION',
 'CAND_ELECTION_YR',
 'CAND_OFFICE_ST',
 'CAND_OFFICE',
 'CAND_OFFICE_DISTRICT',
 'CAND_ICI',
 'CAND_STATUS',
 'CAND_PCC',
 'CAND_ST1',
 'CAND_ST2',
 'CAND_CITY',
 'CAND_ST',
 'CAND_ZIP']

### Based on data from : https://www.fec.gov/data/browse-data/?tab=bulk-data   
This is House/Senate campaign finance data:

In [6]:
data_dir = 'fec_data'
year20 = '20'
year18 = '18'
year16 = '16'
years = [year18, year16] #[year20, year18, year16]


In [7]:
def read_data_in(dataname: str, fields_to_double):
    dfs = {}
    header_row = csv_header(f'{data_dir}/{dataname}/{dataname}_header_file.csv')
    #default to stringtype for ease of loading, then adjust below:
    fields = [*[typ.StructField(h[:], typ.StringType(), True) for h in header_row]]
    schema = typ.StructType(fields)

    for year in years:
        txt_filename = f'{data_dir}/{dataname}/{dataname}{year}.txt'
        df_temp = sc.textFile(txt_filename).map(lambda row: [elem for elem in row.split('|')])
        df = spark.createDataFrame(df_temp, schema)
        for field in fields_to_double:
            df = df.withColumn(field, df[field].cast(DoubleType()))
        dfs[year] = df
    return dfs

**Create dataframes and combine the three files together for analysis:**

### This is the original data that you had read in
The all candidates file contains summary financial information for each candidate who raised or spent money during the period, regardless of when they are up for election.

The file has one record per candidate and shows information about the candidate, total receipts, transfers received from authorized committees, total disbursements, transfers given to authorized committees, cash-on-hand totals, loans and debts, and other financial summary information.

In [8]:
# weball: create campaigns (weball data dir) dataframes
weball_double_fields = [
    'TTL_RECEIPTS', 
    'TTL_INDIV_CONTRIB', 
    'CAND_CONTRIB', 
    'OTHER_POL_CMTE_CONTRIB',
    'POL_PTY_CONTRIB'
]
dfs = read_data_in('weball', weball_double_fields)
df = reduce(DataFrame.unionAll, dfs.values())

The candidate master file contains basic information for each candidate, including:

    Candidates who have filed a Statement of Candidacy (Form 2) for the upcoming election

    Candidates who have active campaign committees without regard to election year

    Candidates who are referenced as a part of a draft committee or a nonconnected committee that registers as supporting or opposing a particular candidate

The file shows the candidate's identification number, candidate’s name, party affiliation, election year, office state, office sought, district, incumbent/challenger status, status as a candidate, name of the candidate’s principal campaign committee, and address.

In [9]:
# cn: The candidate master file contains basic information for each candidate
cn_df = read_data_in('cn', [])

In [34]:
cn_df[year18].show(5)

+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+--------+-----------+---------+--------------------+--------+------------+-------+--------+
|  CAND_ID|           CAND_NAME|CAND_PTY_AFFILIATION|CAND_ELECTION_YR|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|CAND_ICI|CAND_STATUS| CAND_PCC|            CAND_ST1|CAND_ST2|   CAND_CITY|CAND_ST|CAND_ZIP|
+---------+--------------------+--------------------+----------------+--------------+-----------+--------------------+--------+-----------+---------+--------------------+--------+------------+-------+--------+
|H0AK00105|        LAMB, THOMAS|                 NNE|            2020|            AK|          H|                  00|       C|          N|C00607515|1861 W LAKE LUCIL...|        |     WASILLA|     AK|   99654|
|H0AK00113|   TUGATUK, RAY SEAN|                 DEM|            2020|            AK|          H|                  00|       C|          N|         |          P

The committee master file contains one record for each committee registered with the Federal Election Commission. This includes federal political action committees and party committees, campaign committees for presidential, house and senate candidates, as well as groups or organizations who are spending money for or against candidates for federal office.

In [10]:
# cm: committee master file
cm_df = read_data_in('cm', [])
cm_df[year18].show(5)

+---------+--------------------+-----------------+--------------------+-----------+-------------+-------+---------+---------+-------+--------------------+----------------+------+--------------------+---------+
|  CMTE_ID|             CMTE_NM|          TRES_NM|            CMTE_ST1|   CMTE_ST2|    CMTE_CITY|CMTE_ST| CMTE_ZIP|CMTE_DSGN|CMTE_TP|CMTE_PTY_AFFILIATION|CMTE_FILING_FREQ|ORG_TP|    CONNECTED_ORG_NM|  CAND_ID|
+---------+--------------------+-----------------+--------------------+-----------+-------------+-------+---------+---------+-------+--------------------+----------------+------+--------------------+---------+
|C00000018|IRONWORKERS LOCAL...|  STEVEN N GULICK|     43335 W 10 MILE|P O BOX 965|         NOVI|     MI|    48050|        U|      Q|                    |               T|      |IRON WORKERS; INT...|H8TX22313|
|C00000059|  HALLMARK CARDS PAC|        SARAH MOE|          2501 MCGEE|    MD #500|  KANSAS CITY|     MO|    64108|        U|      Q|                 UNK|      

In [11]:
# ccl: This file contains one record for each candidate to committee linkage.
ccl_df = read_data_in('ccl', [])
ccl_df[year18].show(5)

+---------+----------------+---------------+---------+-------+---------+----------+
|  CAND_ID|CAND_ELECTION_YR|FEC_ELECTION_YR|  CMTE_ID|CMTE_TP|CMTE_DSGN|LINKAGE_ID|
+---------+----------------+---------------+---------+-------+---------+----------+
|H0AL02087|            2018|           2018|C00493783|      H|        J|    212430|
|H0AL02087|            2018|           2018|C00462143|      H|        P|    212429|
|H0AL03192|            2020|           2018|C00681452|      H|        P|    221575|
|H0AL05049|            2008|           2018|C00239038|      H|        P|    212431|
|H0AL05163|            2018|           2018|C00464149|      H|        P|    212432|
+---------+----------------+---------------+---------+-------+---------+----------+
only showing top 5 rows



The file has one record per House and Senate campaign committee and shows information about the candidate, total receipts, transfers received from authorized committees, total disbursements, transfers given to authorized committees, cash-on-hand totals, loans and debts, and other financial summary information.

In [12]:
# webl: House Senate campaigns
webl_df = read_data_in('webl', [])
webl_df[year18].show(5)

+---------+-------------------+--------+------+--------------------+------------+---------------+----------+-------------+---------+---------+------------+----------+-----------+---------------+----------------+-------------+-----------------+--------------+--------------------+-------------+-------------+------------+------------+--------------------+----------------------+---------------+----------+-------------+------------+
|  CAND_ID|          CAND_NAME|CAND_ICI|PTY_CD|CAND_PTY_AFFILIATION|TTL_RECEIPTS|TRANS_FROM_AUTH|  TTL_DISB|TRANS_TO_AUTH|  COH_BOP|  COH_COP|CAND_CONTRIB|CAND_LOANS|OTHER_LOANS|CAND_LOAN_REPAY|OTHER_LOAN_REPAY|DEBTS_OWED_BY|TTL_INDIV_CONTRIB|CAND_OFFICE_ST|CAND_OFFICE_DISTRICT|SPEC_ELECTION|PRIM_ELECTION|RUN_ELECTION|GEN_ELECTION|GEN_ELECTION_PRECENT|OTHER_POL_CMTE_CONTRIB|POL_PTY_CONTRIB|CVG_END_DT|INDIV_REFUNDS|CMTE_REFUNDS|
+---------+-------------------+--------+------+--------------------+------------+---------------+----------+-------------+---------+----


Contributions from committees to candidates file description

The contributions from committees to candidates file is a subset of the itemized records (OTH) file and contains each contribution or independent expenditure made by a:

    PAC
    Party committee
    Candidate committee
    Other federal committee

and given to a candidate during the two-year election cycle.

In [13]:
# pas2: Contributions from committees to candidates
pas2_df = read_data_in('pas2', [])
pas2_df[year18].show(5)

+---------+---------+------+---------------+------------------+--------------+---------+--------------------+--------------+-----+--------+--------+----------+--------------+---------------+---------+---------+---------+--------+-------+---------+-------------------+
|  CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|         IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|                NAME|          CITY|STATE|ZIP_CODE|EMPLOYER|OCCUPATION|TRANSACTION_DT|TRANSACTION_AMT| OTHER_ID|  CAND_ID|  TRAN_ID|FILE_NUM|MEMO_CD|MEMO_TEXT|             SUB_ID|
+---------+---------+------+---------------+------------------+--------------+---------+--------------------+--------------+-----+--------+--------+----------+--------------+---------------+---------+---------+---------+--------+-------+---------+-------------------+
|C00390633|        N|    YE|          P2018|201801249090614097|           24K|      CCM|PETE AGUILAR FOR ...|SAN BERNARDINO|   CA|   92423|        |          |      12182017|           2500|C00510


Contributions by individuals file description

The contributions by individuals file contains information for contributions given by individuals. The method used to include contributions in this file has changed over time.
2015 - present: greater than $200

A contribution will be included if:

    The contribution’s election cycle-to-date amount is over $200 for contributions to candidate committees
    The contribution’s calendar year-to-date amount is over $200 for contributions to political action committees (PACs) and party committees.

# The individual files are huge!
## Need to download these files from the Dropbox link and put in the fec_data/indiv folder
- We should make a subset of these to work with in our code
- Then when stuff in getting close to final run on the whole file?

In [14]:
indiv_df = read_data_in('indiv', [])
indiv_df[year16].show(5)

+---------+---------+------+---------------+-----------+--------------+---------+-----------------+------------+-----+---------+----------------+--------------------+--------------+---------------+--------+--------------------+--------+-------+---------+-------------------+
|  CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|  IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|             NAME|        CITY|STATE| ZIP_CODE|        EMPLOYER|          OCCUPATION|TRANSACTION_DT|TRANSACTION_AMT|OTHER_ID|             TRAN_ID|FILE_NUM|MEMO_CD|MEMO_TEXT|             SUB_ID|
+---------+---------+------+---------------+-----------+--------------+---------+-----------------+------------+-----+---------+----------------+--------------------+--------------+---------------+--------+--------------------+--------+-------+---------+-------------------+
|C00088591|        N|    M3|              P|15970306895|            15|      IND|   BURCH, MARY K.|FALLS CHURCH|   VA|220424511|NORTHROP GRUMMAN|VP PROGRAM MANAGE...|      021

# New code that joins tables and counts the number of individual donations each candidate gets
Now that I have this it should be easy to make other stats and aggregate by state, dem/rep, etc

In [ ]:
for year_to_process in years:
    # keep only donations for 'G' the general election
    ind16 = indiv_df[year_to_process].filter(col('TRANSACTION_PGI').startswith('G'))
    # join on 'CMTE_ID', which gives us 'CAND_ID'
    ind16a = ind16.join(ccl_df[year_to_process], on='CMTE_ID', how='inner')
    # count number of individual donations each candidate recieve
    ind16agg = ind16a.groupby(col('CAND_ID')).agg(
        count(lit(1)).alias('numdonat')
    )
    ind16agg.toPandas().to_csv(f'num_donations-{year_to_process}.csv')

In [19]:
# join on 'CAND_ID', which gives us all the info about the candidate
# turns out this step isn't needed, because after aggregating below you do this again.
# ind16b = ind16a.join(cn_df[year_to_process], on='CAND_ID', how='inner')

In [26]:
ind16agg.show(10)

+---------+--------+
|  CAND_ID|numdonat|
+---------+--------+
|H0NH02181|    1652|
|H4NY11138|     124|
|H6AR02245|      10|
|H6CA02234|       1|
|H6IL06117|     882|
|H6KY01110|     133|
|H6MI12272|       2|
|H8UT03089|     127|
|S0FL00338|   24028|
|S6NV00028|     276|
+---------+--------+
only showing top 10 rows



In [ ]:
# join candidates table to above table to get details about candidate
numdonations = ind16agg.join(cn_df[year_to_process], on='CAND_ID', how='inner')

In [29]:
numdonations.show(10)

+---------+--------+--------------------+--------------------+----------------+--------------+-----------+--------------------+--------+-----------+---------+--------------------+--------+-------------+-------+--------+
|  CAND_ID|numdonat|           CAND_NAME|CAND_PTY_AFFILIATION|CAND_ELECTION_YR|CAND_OFFICE_ST|CAND_OFFICE|CAND_OFFICE_DISTRICT|CAND_ICI|CAND_STATUS| CAND_PCC|            CAND_ST1|CAND_ST2|    CAND_CITY|CAND_ST|CAND_ZIP|
+---------+--------+--------------------+--------------------+----------------+--------------+-----------+--------------------+--------+-----------+---------+--------------------+--------+-------------+-------+--------+
|H0NH02181|    1652|  KUSTER, ANN MCLANE|                 DEM|            2016|            NH|          H|                  02|       I|          C|C00462861|         PO BOX 1498|        |      CONCORD|     NH|   03302|
|H4NY11138|     124|    CLARKE, YVETTE D|                 DEM|            2016|            NY|          H|              

## Calculate # of Big Donations feature (> $499)

In [32]:
for year_to_process in years:
    ind16 = indiv_df[year_to_process].filter(col('TRANSACTION_PGI').startswith('G'))
    ind16a = ind16.join(ccl_df[year_to_process], on='CMTE_ID', how='inner')
    ind16agg = ind16a.filter(col('TRANSACTION_AMT') > 499).groupby(col('CAND_ID')).agg(
        count(lit(1)).alias('numbigdonat')
    )
    ind16agg.toPandas().to_csv(f'num_big_donations-{year_to_process}.csv')

## Calculate # of Out of State donations

In [33]:
for year_to_process in years:
    ind16 = indiv_df[year_to_process].filter(col('TRANSACTION_PGI').startswith('G'))
    ind16a = ind16.join(ccl_df[year_to_process], on='CMTE_ID', how='inner')
    ind16b = ind16a.join(cn_df[year_to_process], on='CAND_ID', how='inner')
    ind16agg = ind16b.filter(col('CAND_ST') != col('STATE')).groupby(col('CAND_ID')).agg(
        count(lit(1)).alias('numoutofstatedonat')
    )
    ind16agg.toPandas().to_csv(f'num_out_of_state_donations-{year_to_process}.csv')

## Calculate Average & Total Donations

In [16]:
for year_to_process in years:
    ind16 = indiv_df[year_to_process].filter(col('TRANSACTION_PGI').startswith('G'))
    ind16a = ind16.join(ccl_df[year_to_process], on='CMTE_ID', how='inner')
    ind16agg = ind16a.groupby(col('CAND_ID')).agg(
        avg(col('TRANSACTION_AMT')).alias('avgdonation'),
        sum(col('TRANSACTION_AMT')).alias('sumdonation')
    )
    ind16agg.toPandas().to_csv(f'avgsum_donation-{year_to_process}.csv')

### For Visual aid

In [15]:
for year_to_process in years:
    ind16 = indiv_df[year_to_process].filter(col('TRANSACTION_PGI').startswith('G'))
    ind16a = ind16.join(ccl_df[year_to_process], on='CMTE_ID', how='inner')
    ind16b = ind16a.join(cn_df[year_to_process], on='CAND_ID', how='inner')
    ind16agg = ind16b.groupby(col('CAND_ST')).agg(
        avg(col('TRANSACTION_AMT')).alias('avgdonation'),
        sum(col('TRANSACTION_AMT')).alias('sumdonation')
    )
    ind16agg.toPandas().to_csv(f'state-avgsum_donation-{year_to_process}.csv')

## ------------------------------
## Everything after this point is exploratory work we did very early on

In [29]:
df.count()

10476

---
**Number of columns:**

In [30]:
len(df.columns)

30

---
**Statistical summary of response variable:**

Our statistical summary will be based on whether a candidate won or lost the relevant political race.  
  
We are still gathering and joining that data to this set.

---
**Statistical summary of potential predictor variables:**

Total receipts -

In [31]:
df.select('TTL_RECEIPTS').describe().show()

+-------+-------------------+
|summary|       TTL_RECEIPTS|
+-------+-------------------+
|  count|              10476|
|   mean| 1594327.4810843852|
| stddev|4.960967162034303E7|
|    min|          -674132.5|
|    max|      4.824617973E9|
+-------+-------------------+



In [54]:
df.select('TTL_RECEIPTS').describe().show()

+-------+-------------------+
|summary|       TTL_RECEIPTS|
+-------+-------------------+
|  count|               7149|
|   mean| 2266553.4508938333|
| stddev|6.004019782547508E7|
|    min|                0.0|
|    max|      4.824617973E9|
+-------+-------------------+



Contributions by individuals -

In [55]:
df.select('TTL_INDIV_CONTRIB').describe().show()

+-------+-------------------+
|summary|  TTL_INDIV_CONTRIB|
+-------+-------------------+
|  count|               7149|
|   mean|  3570474.139283814|
| stddev|2.231747534710199E8|
|    min|                0.0|
|    max|    1.8853982587E10|
+-------+-------------------+



Contributions by candidates -

In [57]:
df.select('CAND_CONTRIB').describe().show()

+-------+-------------------+
|summary|       CAND_CONTRIB|
+-------+-------------------+
|  count|               7149|
|   mean|  621843.8302853543|
| stddev|3.607460293607057E7|
|    min|                0.0|
|    max|      2.831281203E9|
+-------+-------------------+



Contributions from party committees -

In [64]:
df.select('POL_PTY_CONTRIB').describe().show()

+-------+------------------+
|summary|   POL_PTY_CONTRIB|
+-------+------------------+
|  count|              7149|
|   mean|1594.4136438662752|
| stddev| 37564.26071346217|
|    min|               0.0|
|    max|         3100000.0|
+-------+------------------+



Contributions from other political committees -

In [65]:
df.select('OTHER_POL_CMTE_CONTRIB').describe().show()

+-------+----------------------+
|summary|OTHER_POL_CMTE_CONTRIB|
+-------+----------------------+
|  count|                  7149|
|   mean|     449703.3054748916|
| stddev|  2.2751089972151406E7|
|    min|                   0.0|
|    max|           1.9235003E9|
+-------+----------------------+



Candidate status (C = Challenger, O = Open, I = Incumbent) -

In [61]:
#some data cleaning to do for the blanks
df.groupby('CAND_ICI').count().orderBy('count', ascending = False).show()

+--------+-----+
|CAND_ICI|count|
+--------+-----+
|       C| 3857|
|       O| 1779|
|       I| 1441|
|        |   72|
+--------+-----+



Candidate party affiliation (count) -

In [41]:
df.groupby('CAND_PTY_AFFILIATION').count().orderBy('count', ascending = False).show()

+--------------------+-----+
|CAND_PTY_AFFILIATION|count|
+--------------------+-----+
|                 DEM| 3227|
|                 REP| 3193|
|                 IND|  272|
|                 LIB|  136|
|                 GRE|   55|
|                 NPA|   37|
|                 DFL|   36|
|                 OTH|   35|
|                 NNE|   32|
|                 UNK|   26|
|                  UN|   23|
|                 CON|   14|
|                   W|    9|
|                 NON|    7|
|                 IDP|    5|
|                 NOP|    5|
|                 AMP|    3|
|                 PPY|    3|
|                 SEP|    3|
|                 UNI|    3|
+--------------------+-----+
only showing top 20 rows



Candidate state (count) - 

In [59]:
df.groupby('CAND_OFFICE_ST').count().orderBy('count', ascending = False).show()

+--------------+-----+
|CAND_OFFICE_ST|count|
+--------------+-----+
|            CA|  695|
|            TX|  604|
|            00|  513|
|            FL|  486|
|            NY|  364|
|            PA|  263|
|            GA|  235|
|            IL|  228|
|            NC|  224|
|            MI|  189|
|            OH|  183|
|            VA|  171|
|            AZ|  170|
|            NJ|  170|
|            IN|  148|
|            TN|  147|
|            MD|  147|
|            WA|  132|
|            MN|  129|
|            CO|  125|
+--------------+-----+
only showing top 20 rows



In [66]:
df.select('CAND_NAME', 
          'CAND_OFFICE_ST',
          'CAND_PTY_AFFILIATION', 
          'CAND_ICI', 
          'TTL_RECEIPTS',
          'CAND_CONTRIB',    
          'TTL_INDIV_CONTRIB',
          'POL_PTY_CONTRIB',
          'OTHER_POL_CMTE_CONTRIB').show(5)

+--------------------+--------------+--------------------+--------+------------+------------+-----------------+---------------+----------------------+
|           CAND_NAME|CAND_OFFICE_ST|CAND_PTY_AFFILIATION|CAND_ICI|TTL_RECEIPTS|CAND_CONTRIB|TTL_INDIV_CONTRIB|POL_PTY_CONTRIB|OTHER_POL_CMTE_CONTRIB|
+--------------------+--------------+--------------------+--------+------------+------------+-----------------+---------------+----------------------+
|     YOUNG, DONALD E|            AK|                 REP|       I|  1362383.63|         0.0|        637025.31|            0.0|             584444.63|
|       GALVIN, ALYSE|            AK|                 IND|       C|  2266364.63|     3394.63|        2116292.8|            0.0|              109350.0|
|     AVERHART, JAMES|            AL|                 DEM|       O|    50126.74|         0.0|         23281.74|            0.0|                   0.0|
|    GARDNER, KIANI A|            AL|                 DEM|       O|   118661.85|      764.97| 

In [67]:
!jupyter nbconvert --to pdf Exploratory_Work-Group5.ipynb

[NbConvertApp] Converting notebook Exploratory_Work-Group5.ipynb to pdf
[NbConvertApp] Writing 43970 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 51768 bytes to Exploratory_Work-Group5.pdf
